# Events location processing

After scraping the ted website with `scraper.py`, we still have missing events. Let's analyze them and fill in as much missing data as possible.

In [105]:
%matplotlib inline
import numpy as np
import pandas as pd

# Uncomment this if you want to see all rows and columns when displaying a pandas object
# pd.set_option("display.max_rows", None, "display.max_columns", None)

In [106]:
events = pd.read_csv('data/locations_scraped.csv')
events = events.fillna('')

print('Total number of events: {0}'.format(events.shape[0]))
print('Total number of events with location: {0}'.format(events[events['city'].notnull()].shape[0]))

Total number of events: 356
Total number of events with location: 356


In [107]:
events[events['city'] != ''].shape[0]

180

## Number of missing locations per event type

In [108]:
no_location = events[events['city'] == '']
location = events[events['city'] != '']

In [109]:
events_type = events['event_type'].unique()

for event in events_type:
    a = no_location[no_location['event_type'] == event].shape[0]
    b = location[location['event_type'] == event].shape[0]
    print('For {0}, {1} have no location while {2} have location'.format(event, a, b))

For main, 21 have no location while 1 have location
For external, 40 have no location while 0 have location
For satellite, 45 have no location while 0 have location
For global, 12 have no location while 0 have location
For med, 8 have no location while 0 have location
For salon, 15 have no location while 0 have location
For women, 4 have no location while 0 have location
For youth, 5 have no location while 0 have location
For tedx, 26 have no location while 179 have location


main: all main events are in 3 locations (Monterey, Vancouver or Long Beach) <br />
external: these are not exactly ted events, just talks which are ted worthy <br />
For the others we are going to manually input the data where we can, if the location is mentioned in the event name: TED@Bangalore, TED@London

In [110]:
# Loading a csv with the major cities
major_cities = pd.read_csv('data/world-cities_csv.csv')
major_cities['city'] = major_cities['name']
major_cities['name'] = major_cities['name'].apply(lambda x: x.replace(' ', '').lower())
major_cities.head()

,name,country,subcountry,geonameid,city
0,lesescaldes,Andorra,Escaldes-Engordany,3040051,les Escaldes
1,andorralavella,Andorra,Andorra la Vella,3041563,Andorra la Vella
2,ummalqaywayn,United Arab Emirates,Umm al Qaywayn,290594,Umm al Qaywayn
3,rasal-khaimah,United Arab Emirates,Raʼs al Khaymah,291074,Ras al-Khaimah
4,khawrfakkān,United Arab Emirates,Ash Shāriqah,291696,Khawr Fakkān


In [111]:
for i, row in events.iterrows():
    event_name = row['event'].replace(' ', '').lower()
    if row['city'] == '':
        # Names should be > 4 letters long
        for x in range(len(event_name)):
            for y in range(x+1, len(event_name)):
                substring = event_name[x:y+1]
                if substring in major_cities['name'].values:
                    city_row = major_cities[major_cities['name'] == substring]
                    events.loc[i, 'city'] = city_row['city'].values[0]
                    events.loc[i, 'country'] = city_row['country'].values[0]

In [112]:
# Recompute
no_location = events[events['city'] == '']
location = events[events['city'] != '']
events_type = events['event_type'].unique()

for event in events_type:
    a = no_location[no_location['event_type'] == event].shape[0]
    b = location[location['event_type'] == event].shape[0]
    print('For {0}, {1} have no location while {2} have location'.format(event, a, b))

For main, 21 have no location while 1 have location
For external, 19 have no location while 21 have location
For satellite, 20 have no location while 25 have location
For global, 9 have no location while 3 have location
For med, 8 have no location while 0 have location
For salon, 0 have no location while 15 have location
For women, 4 have no location while 0 have location
For youth, 5 have no location while 0 have location
For tedx, 14 have no location while 191 have location


### Women

In [113]:
no_location[no_location['event_type'] == 'women']

,event,event_type,avg_views,avg_speed_of_speech,avg_duration,film_date,nof_talks,talks,event_id,city,country
136,TEDWomen 2010,women,1.298086e+06,144.759908,12.066667,2010-02-12,34,"[826, 827, 828, 829, 831, 834, 835, 839, 842, ...",135,,
137,TEDWomen 2013,women,2.504863e+06,148.644765,12.944444,2013-04-12,9,"[1639, 1647, 1649, 1651, 1659, 1663, 1670, 167...",136,,
138,TEDWomen 2015,women,1.449902e+06,133.743150,12.637500,2015-05-27,28,"[1991, 1995, 1996, 1997, 2001, 2006, 2008, 201...",137,,
139,TEDWomen 2016,women,1.340357e+06,142.858163,14.310000,2016-10-26,25,"[2320, 2321, 2325, 2330, 2334, 2338, 2341, 234...",138,,


In [114]:
# Manually fill in location for women (from ted.com)
events.loc[events['event'] == 'TEDWomen 2010', 'city'] = 'Washington'
events.loc[events['event'] == 'TEDWomen 2010', 'country'] = 'United States'
events.loc[events['event'] == 'TEDWomen 2013', 'city'] = 'San Francisco'
events.loc[events['event'] == 'TEDWomen 2013', 'country'] = 'United States'
events.loc[events['event'] == 'TEDWomen 2015', 'city'] = 'Monterey'
events.loc[events['event'] == 'TEDWomen 2015', 'country'] = 'United States'
events.loc[events['event'] == 'TEDWomen 2016', 'city'] = 'San Francisco'
events.loc[events['event'] == 'TEDWomen 2016', 'country'] = 'United States'

### Youth

In [115]:
no_location[no_location['event_type'] == 'youth']

,event,event_type,avg_views,avg_speed_of_speech,avg_duration,film_date,nof_talks,talks,event_id,city,country
140,TEDYouth 2011,youth,1.343913e+06,120.817798,7.600000,2011-11-19,3,"[1157, 1379, 1395]",139,,
141,TEDYouth 2012,youth,2.749860e+05,0.000000,6.466667,2012-11-17,1,[1467],140,,
142,TEDYouth 2013,youth,9.628063e+05,177.753402,6.705556,2013-11-16,3,"[1641, 1668, 1688]",141,,
143,TEDYouth 2014,youth,1.206283e+06,168.505825,6.540476,2014-04-11,7,"[1888, 1893, 1900, 1911, 1912, 1914, 1964]",142,,
144,TEDYouth 2015,youth,1.771106e+06,146.717383,6.426667,2015-11-14,5,"[2099, 2100, 2101, 2119, 2150]",143,,


In [116]:
# Manually fill in location for youth (from ted.com)
events.loc[events['event'] == 'TEDYouth 2011', 'city'] = 'New York'
events.loc[events['event'] == 'TEDYouth 2011', 'country'] = 'United States'
events.loc[events['event'] == 'TEDYouth 2012', 'city'] = 'New York'
events.loc[events['event'] == 'TEDYouth 2012', 'country'] = 'United States'
events.loc[events['event'] == 'TEDYouth 2013', 'city'] = 'New Orleans'  # !! Different than the rest
events.loc[events['event'] == 'TEDYouth 2013', 'country'] = 'United States'
events.loc[events['event'] == 'TEDYouth 2014', 'city'] = 'New York'
events.loc[events['event'] == 'TEDYouth 2014', 'country'] = 'United States'
events.loc[events['event'] == 'TEDYouth 2015', 'city'] = 'New York'
events.loc[events['event'] == 'TEDYouth 2015', 'country'] = 'United States'

### Med

In [117]:
no_location[no_location['event_type'] == 'med']

,event,event_type,avg_views,avg_speed_of_speech,avg_duration,film_date,nof_talks,talks,event_id,city,country
109,TEDMED 2009,med,2.246144e+06,163.313356,17.571212,2009-10-10,11,"[596, 598, 602, 609, 611, 617, 621, 636, 639, ...",108,,
110,TEDMED 2010,med,4.763132e+05,167.484619,18.775000,2010-10-10,4,"[850, 855, 858, 877]",109,,
111,TEDMED 2011,med,7.720503e+05,148.891851,14.226190,2011-10-24,7,"[1077, 1084, 1089, 1096, 1100, 1122, 1128]",110,,
112,TEDMED 2012,med,1.351933e+06,152.173905,13.948333,2012-04-15,10,"[1252, 1258, 1265, 1271, 1282, 1336, 1342, 134...",111,,
113,TEDMED 2013,med,1.681900e+06,140.529529,14.962500,2013-04-16,8,"[1517, 1533, 1557, 1591, 1599, 1602, 1650, 2016]",112,,
114,TEDMED 2014,med,1.743996e+06,135.038771,13.826667,2014-09-09,10,"[1832, 1840, 1844, 1849, 1853, 1860, 1863, 186...",113,,
115,TEDMED 2015,med,2.053347e+06,139.673060,11.700000,2015-11-18,8,"[2131, 2138, 2147, 2151, 2197, 2268, 2301, 2305]",114,,
116,TEDMED 2016,med,1.241131e+06,128.910646,13.445000,2016-11-30,10,"[2374, 2403, 2419, 2424, 2434, 2446, 2452, 247...",115,,


In [118]:
# Manually fill in location for med (from wikipedia)
events.loc[events['event'] == 'TEDMED 2009', 'city'] = 'San Diego'
events.loc[events['event'] == 'TEDMED 2009', 'country'] = 'United States'
events.loc[events['event'] == 'TEDMED 2010', 'city'] = 'San Diego'
events.loc[events['event'] == 'TEDMED 2010', 'country'] = 'United States'
events.loc[events['event'] == 'TEDMED 2011', 'city'] = 'Washington'
events.loc[events['event'] == 'TEDMED 2011', 'country'] = 'United States'
events.loc[events['event'] == 'TEDMED 2012', 'city'] = 'Washington'
events.loc[events['event'] == 'TEDMED 2012', 'country'] = 'United States'
events.loc[events['event'] == 'TEDMED 2013', 'city'] = 'Washington'
events.loc[events['event'] == 'TEDMED 2013', 'country'] = 'United States'
events.loc[events['event'] == 'TEDMED 2014', 'city'] = 'Washington'
events.loc[events['event'] == 'TEDMED 2014', 'country'] = 'United States'
events.loc[events['event'] == 'TEDMED 2015', 'city'] = 'Washington'
events.loc[events['event'] == 'TEDMED 2015', 'country'] = 'United States'
events.loc[events['event'] == 'TEDMED 2016', 'city'] = 'Palm Springs'
events.loc[events['event'] == 'TEDMED 2016', 'country'] = 'United States'

### Global

In [119]:
no_location[no_location['event_type'] == 'global']

,event,event_type,avg_views,avg_speed_of_speech,avg_duration,film_date,nof_talks,talks,event_id,city,country
95,TEDGlobal 2005,global,1.639063e+06,166.480970,18.496154,2005-01-07,26,"[19, 25, 26, 30, 32, 33, 34, 35, 36, 42, 50, 5...",94,,
96,TEDGlobal 2007,global,5.621498e+05,131.862544,15.073457,2007-04-06,27,"[127, 128, 129, 130, 131, 132, 133, 134, 135, ...",95,,
97,TEDGlobal 2009,global,1.679021e+06,148.224575,13.520000,2009-07-07,65,"[491, 492, 494, 495, 496, 497, 498, 499, 500, ...",96,,
98,TEDGlobal 2010,global,1.338910e+06,153.532743,14.411515,2010-07-13,55,"[728, 729, 730, 731, 732, 733, 734, 736, 740, ...",97,,
99,TEDGlobal 2011,global,1.717761e+06,149.172750,13.213480,2011-07-13,68,"[978, 979, 980, 981, 982, 983, 984, 985, 986, ...",98,,
100,TEDGlobal 2012,global,2.072436e+06,148.676288,12.854524,2012-06-19,70,"[1273, 1274, 1275, 1276, 1277, 1280, 1281, 128...",99,,
101,TEDGlobal 2013,global,2.584163e+06,144.112618,13.044192,2013-06-13,66,"[1523, 1524, 1525, 1526, 1527, 1528, 1529, 153...",100,,
102,TEDGlobal 2014,global,1.316167e+06,139.924767,12.816667,2014-02-10,51,"[1837, 1838, 1839, 1842, 1843, 1846, 1848, 185...",101,,
103,TEDGlobal 2017,global,5.560813e+05,118.419158,12.933333,2017-08-27,3,"[2535, 2542, 2544]",102,,


In [120]:
# Manually fill in location for gloabl (from ted.com)
events.loc[events['event'] == 'TEDGlobal 2005', 'city'] = 'Oxford'
events.loc[events['event'] == 'TEDGlobal 2005', 'country'] = 'United Kingdom'
events.loc[events['event'] == 'TEDGlobal 2007', 'city'] = 'Arusha'
events.loc[events['event'] == 'TEDGlobal 2007', 'country'] = 'Tanzania'
events.loc[events['event'] == 'TEDGlobal 2009', 'city'] = 'Oxford'
events.loc[events['event'] == 'TEDGlobal 2009', 'country'] = 'United Kingdom'
events.loc[events['event'] == 'TEDGlobal 2010', 'city'] = 'Oxford'
events.loc[events['event'] == 'TEDGlobal 2010', 'country'] = 'United Kingdom'
events.loc[events['event'] == 'TEDGlobal 2011', 'city'] = 'Edinburgh'
events.loc[events['event'] == 'TEDGlobal 2011', 'country'] = 'United Kingdom'
events.loc[events['event'] == 'TEDGlobal 2012', 'city'] = 'Edinburgh'
events.loc[events['event'] == 'TEDGlobal 2012', 'country'] = 'United Kingdom'
events.loc[events['event'] == 'TEDGlobal 2013', 'city'] = 'Edinburgh'
events.loc[events['event'] == 'TEDGlobal 2013', 'country'] = 'United Kingdom'
events.loc[events['event'] == 'TEDGlobal 2014', 'city'] = 'Rio de Janeiro'
events.loc[events['event'] == 'TEDGlobal 2014', 'country'] = 'Brazil'
events.loc[events['event'] == 'TEDGlobal 2017', 'city'] = 'Arusha'
events.loc[events['event'] == 'TEDGlobal 2017', 'country'] = 'Tanzania'

### Tedx

In [121]:
no_location[no_location['event_type'] == 'tedx']

,event,event_type,avg_views,avg_speed_of_speech,avg_duration,film_date,nof_talks,talks,event_id,city,country
146,TEDxAmazonia,tedx,9.849730e+05,144.694981,21.583333,2010-08-11,1,[1824],145,,
181,TEDxChange,tedx,1.032390e+06,159.632886,17.829167,2010-09-20,4,"[780, 784, 787, 1202]",180,,
192,TEDxDU 2010,tedx,9.001055e+05,133.290068,12.400000,2010-05-13,2,"[809, 817]",191,,
193,TEDxDU 2011,tedx,3.693630e+05,146.745914,11.216667,2011-05-13,1,[1092],192,,
201,TEDxEQChCh,tedx,2.829484e+06,124.373178,17.150000,2012-01-09,1,[1386],200,,
235,TEDxKrakow,tedx,8.004180e+05,161.382320,12.633333,2011-11-10,2,"[1081, 1203]",234,,
251,TEDxMidwest,tedx,2.479920e+06,175.663266,17.361111,2010-10-09,3,"[819, 822, 836]",250,,
266,TEDxNorrkoping,tedx,6.569493e+06,100.161290,10.333333,2014-11-11,1,[1885],265,,
290,TEDxRC2,tedx,4.962260e+05,135.811914,14.983333,2011-11-11,3,"[1099, 1108, 1147]",289,,
295,TEDxSF,tedx,3.658158e+06,103.645656,9.783333,2011-11-06,1,[1384],294,,


In [ ]:
# Manually fill in location for tedx (from ted.com)
events.loc[events['event'] == 'TEDxAmazonia', 'city'] = 'Manaus'
events.loc[events['event'] == 'TEDxAmazonia', 'country'] = 'Brazil'
events.loc[events['event'] == 'TEDxChange', 'city'] = 'New Yrok'
events.loc[events['event'] == 'TEDxChange', 'country'] = 'United States'
events.loc[events['event'] == 'TEDxDU 2010', 'city'] = 'Denver'
events.loc[events['event'] == 'TEDxDU 2010', 'country'] = 'United States'
events.loc[events['event'] == 'TEDxDU 2011', 'city'] = 'Denver'
events.loc[events['event'] == 'TEDxDU 2011', 'country'] = 'United Kingdom'
events.loc[events['event'] == 'TEDxEQChCh', 'city'] = 'Christchurch'
events.loc[events['event'] == 'TEDxEQChCh', 'country'] = 'New Zealand'
events.loc[events['event'] == 'TEDxKrakow', 'city'] = 'Krakow'
events.loc[events['event'] == 'TEDxKrakow', 'country'] = 'Poland'
events.loc[events['event'] == 'TEDxMidwest', 'city'] = 'Chicago'
events.loc[events['event'] == 'TEDxMidwest', 'country'] = 'United States'
events.loc[events['event'] == 'TEDxNorrkoping', 'city'] = 'Norrkoping'
events.loc[events['event'] == 'TEDxNorrkoping', 'country'] = 'Sweden'
events.loc[events['event'] == 'TEDxRC2', 'city'] = 'Geneve'
events.loc[events['event'] == 'TEDxRC2', 'country'] = 'Switzerland'
events.loc[events['event'] == 'TEDxSF', 'city'] = 'San Francisco'
events.loc[events['event'] == 'TEDxSF', 'country'] = 'United States'
events.loc[events['event'] == 'TEDxUF', 'city'] = 'Gainesville'
events.loc[events['event'] == 'TEDxUF', 'country'] = 'United States'
events.loc[events['event'] == 'TEDxUM', 'city'] = 'Oxford'
events.loc[events['event'] == 'TEDxUM', 'country'] = 'United States'
events.loc[events['event'] == 'TEDxWomen 2011', 'city'] = 'New York'
events.loc[events['event'] == 'TEDxWomen 2011', 'country'] = 'United States'
events.loc[events['event'] == 'TEDxWomen 2012', 'city'] = 'Washington'
events.loc[events['event'] == 'TEDxWomen 2012', 'country'] = 'United States'

### Main

In [ ]:
no_location[no_location['event_type'] == 'main']

In [123]:
# Manually fill in location for main (from ted.com)
# Most are at Monterey (12) so we fill them all at Monterey
events.loc[events['event_type'] == 'main', 'city'] = 'Monterey'
events.loc[events['event_type'] == 'main', 'country'] = 'United States'

# Some of them are in Long Beach (5)
long_beach = ['TED2013', 'TED2012', 'TED2011', 'TED2010', 'TED2009']
events.loc[events['event'].isin(long_beach), 'city'] = 'Long Beach'
events.loc[events['event'].isin(long_beach), 'country'] = 'United States'

# Some of them are in Vancouver (4)
vancouver = ['TED2017', 'TED2016', 'TED2015', 'TED2014']
events.loc[events['event'].isin(vancouver), 'city'] = 'Vancouver'
events.loc[events['event'].isin(vancouver), 'country'] = 'Canada'

In [128]:
# Recompute
no_location = events[events['city'] == '']
location = events[events['city'] != '']
events_type = events['event_type'].unique()

for event in events_type:
    a = no_location[no_location['event_type'] == event].shape[0]
    b = location[location['event_type'] == event].shape[0]
    print('For {0}, {1} have no location while {2} have location'.format(event, a, b))

For main, 0 have no location while 22 have location
For external, 19 have no location while 21 have location
For satellite, 20 have no location while 25 have location
For global, 0 have no location while 12 have location
For med, 0 have no location while 8 have location
For salon, 0 have no location while 15 have location
For women, 0 have no location while 4 have location
For youth, 0 have no location while 5 have location
For tedx, 0 have no location while 205 have location


We will **not** include location data for external and satellite event types.

## Longitude and latitude

Steps:
1. get all cities with countries in a separate file
2. input the file here https://www.gpsvisualizer.com/geocoder/ (source: MapQuest, easier to use, must have an account before)
3. get the output and add number of events, save them in resources/cities.csv

In [140]:
# 1
events['address'] = events['city'] + ', ' + events['country']
all_address = events['address'].unique()
file = open('temp.txt', 'w')
for address in all_address:
    file.write(address + '\n')
file.close()

In [144]:
# 2
# Fill in the name of the file
gps_visualizer_csv = 'data/cities_coordinates.csv'

In [148]:
# 3
# This file has one address per line, the name field represents the address
address_coordinates = pd.read_csv(gps_visualizer_csv)
events_coordinates = events.merge(address_coordinates, left_on='address', right_on='name')
events_coordinates.head()

,event,event_type,avg_views,avg_speed_of_speech,avg_duration,film_date,nof_talks,talks,event_id,city,country,address,latitude,longitude,name,desc,color,source,precision
0,TEDxSingapore,main,1798071.25,142.631379,13.091422,2011-02-02,68,"[614, 615, 616, 618, 620, 622, 624, 626, 628, ...",60,Monterey,United States,"Monterey, United States",36.596808,-121.897127,"Monterey, United States","Monterey, Monterey County, CA, US",NaN,MapQuest,city/town
1,TEDWomen 2015,women,1449901.50,133.743150,12.637500,2015-05-27,28,"[1991, 1995, 1996, 1997, 2001, 2006, 2008, 201...",137,Monterey,United States,"Monterey, United States",36.596808,-121.897127,"Monterey, United States","Monterey, Monterey County, CA, US",NaN,MapQuest,city/town
2,TEDxMonterey,tedx,166835.00,0.000000,11.616667,2012-04-18,1,[1237],253,Monterey,United States,"Monterey, United States",36.596808,-121.897127,"Monterey, United States","Monterey, Monterey County, CA, US",NaN,MapQuest,city/town
3,Arbejdsglaede Live,external,971594.00,166.208791,18.200000,2009-05-05,1,[632],1,Ede,Netherlands,"Ede, Netherlands",52.045827,5.670201,"Ede, Netherlands","Ede, NL",NaN,MapQuest,city/town
4,Bowery Poetry Club,external,676741.00,0.000000,3.033333,2005-12-11,1,[601],3,Wer,India,"Wer, India",28.632430,77.218790,"Wer, India",IN,NaN,MapQuest,country


In [155]:
cities_coordinates = events_coordinates.groupby(['latitude', 'longitude', 'desc'], as_index='False')['event'].count().reset_index(name='count')
cities_coordinates['desc'] = cities_coordinates['desc'].apply(lambda x: x[:x.find(',')])

In [157]:
cities_coordinates

,latitude,longitude,desc,count
0,-43.530955,172.636646,Christchurch,2
1,-35.282071,149.128667,Canberra,1
2,-34.612869,-58.445979,Buenos Aires,1
3,-34.055000,18.475600,Retreat,2
4,-33.854816,151.216454,Sydney,2
...,...,...,...,...
180,56.878718,14.809439,Växjö,1
181,58.460278,8.766667,Arendal,1
182,58.590913,16.190348,Norrköping,2
183,59.913269,10.739111,Oslo,1


In [158]:
cities_coordinates.to_csv('resources/cities.csv', index=False, header=True)